<a href="https://colab.research.google.com/github/ttury/deedee/blob/main/DeeDee4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# transformer.py 파일 업로드 필요

from google.colab import files

src = list(files.upload().values())[0]
open('transformer.py','wb').write(src)

Saving transformer.py to transformer.py


13710

In [ ]:
open('./drive/MyDrive/packages/transformer.py', 'wb')

FileNotFoundError: ignored

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import *

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

print(len(train_data))
print(train_data.isnull().sum())
train_data.head()

11823
Q        0
A        0
label    0
dtype: int64


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = list()
for sentence in train_data['Q']:
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip() # 양쪽 공백 제거
  questions.append(sentence)

answers = list()
for sentence in train_data['A']:
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip() # 양쪽 공백 제거
  answers.append(sentence)

print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


In [4]:
corpus = questions + answers

with open('corpus.txt','w',encoding='UTF-8') as f:
    for row in corpus:
        f.write(row+'\n')

In [5]:
for i in range(5):
  print(corpus[i])

12시 땡 !
1지망 학교 떨어졌어
3박4일 놀러가고 싶다
3박4일 정도 놀러가고 싶다
PPL 심하네


In [6]:
# 서브워드텍스트인코더를 이용해 단어 사전 생성

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13
)

# 단어 사전에 인덱스 2개 추가(디코더의 입력에 사용하는 SOS, EOS 토큰)
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

print('SOS 토큰 번호 :', START_TOKEN)
print('EOS 토큰 번호 :', END_TOKEN)
print('단어 집합의 크기 :', VOCAB_SIZE)
# 패딩 토큰 + 8179번 토큰 = 단어 집합의 크기

SOS 토큰 번호 : [8178]
EOS 토큰 번호 : [8179]
단어 집합의 크기 : 8180


In [7]:
def make_subword_corpus(corpus):
  subword_corpus = []
  for sent in corpus:
    encoded_sent = tokenizer.encode(sent)
    decoded_sent = []
    for index in encoded_sent:
      decoded_sent.append(str(tokenizer._id_to_subword(index-1)))
    subword_corpus.append(decoded_sent)
  return subword_corpus

subword_corpus = make_subword_corpus(corpus)

In [45]:
subword_corpus[:5]

[['12', '시_', '땡', ' !'],
 ["b'1'", '지', '망', "b' '", '학교_', '떨어졌어'],
 ["b'3'", '박', '4일', "b' '", '놀러가고_', '싶다'],
 ["b'3'", '박', '4일', "b' '", '정도_', '놀러가고_', '싶다'],
 ["b'P'", "b'P'", "b'L'", "b' '", '심하네']]

In [8]:
pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.1 MB/s 


In [9]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(subword_corpus, window=3)
glove = Glove(no_components=256, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [18]:
print(glove.most_similar("실패"))

[('문제가_', 0.7553508662533543), ('증_', 0.7552565071543065), ('주면_', 0.7536235740697377), ('사람마다_', 0.7526890297180965)]


In [19]:
# 서브워드텍스트인코더 토크나이저의 encode()를 사용하여 정수 인코딩

sample_string = questions[20]

tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

for token in tokenized_string:
  print ('{} ----> {}'.format(token, tokenizer.decode([token])))
  # decode의 경우 리스트를 인자로 받는다
  # 한 어절이 정수 인코딩 후에는 여러 서브워드로 맵핑될 수 있음

정수 인코딩 후의 문장 : [5766, 611, 3509, 141, 685, 3747, 849]
기존 문장 : 가스비 비싼데 감기 걸리겠어
5766 ----> 가스
611 ----> 비 
3509 ----> 비싼
141 ----> 데 
685 ----> 감기 
3747 ----> 걸리
849 ----> 겠어


In [20]:
dir(glove)
# glove.dictionary
len(glove.word_vectors)

8024

In [25]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

word2vec = Word2Vec(sentences = subword_corpus, size = 256, window = 3, min_count = 0, workers = 4, sg = 0)
word2vec.save('word2vec2') # 모델 저장
loaded_word2vec = Word2Vec.load("word2vec2") # 모델 로드

In [21]:
import gensim
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import datapath

print(len(glove.word_vectors[0]))
embedding_matrix = np.zeros((VOCAB_SIZE, 256))

def get_vector(word):
  if word in glove.dictionary:
    index = glove.dictionary[word]
    return glove.word_vectors[index]
  else:
    return None

for index in range(VOCAB_SIZE - 3):
  subword = tokenizer._id_to_subword(index)
  temp = get_vector(subword)
  if temp is not None:
    embedding_matrix[index] = temp

256


In [24]:
import pandas as pd

dataframe = pd.DataFrame(embedding_matrix)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.374902,-0.220930,0.346664,0.190356,-0.135725,-0.191327,0.130512,0.222599,0.277908,-0.103534,...,-0.423148,-0.520842,-0.092600,0.243741,0.292972,-0.388110,-0.276810,-0.359022,-0.155439,0.091506
1,0.078889,0.128243,0.020384,-0.002038,-0.093152,-0.277866,0.020054,0.275233,0.232757,0.139142,...,-0.240255,-0.105939,0.138204,0.158927,0.164136,-0.231693,-0.209784,-0.109231,-0.015964,-0.047505
2,0.097408,0.193976,-0.154381,-0.130508,0.162438,-0.133396,0.090741,0.152188,0.113723,0.083148,...,-0.197661,-0.125690,0.033860,0.030857,0.167280,-0.059977,-0.136021,-0.226841,0.064469,-0.044220
3,0.072457,0.380852,-0.242994,-0.105706,0.210669,-0.075455,0.110732,0.169382,0.027043,-0.031713,...,-0.148186,-0.037606,-0.058978,0.005882,0.154388,-0.048058,-0.102507,-0.197358,0.230151,-0.088693
4,0.034402,-0.071904,-0.000665,-0.071989,-0.006545,-0.089077,0.012757,0.043041,0.036165,0.077681,...,-0.123474,-0.124995,0.061137,0.073845,0.069021,-0.035103,-0.004320,-0.084238,-0.075939,0.016717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8177,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
MAX_LENGTH = 40

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sent1, sent2) in zip(inputs, outputs):
    sent1 = START_TOKEN + tokenizer.encode(sent1) + END_TOKEN
    sent2 = START_TOKEN + tokenizer.encode(sent2) + END_TOKEN

    tokenized_inputs.append(sent1)
    tokenized_outputs.append(sent2)
  
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post') # 뒤를 패딩
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [27]:
print('Q 데이터의 크기(shape) :', questions.shape)
print('A 데이터의 크기(shape) :', answers.shape)
print()
print(questions[0])
print(answers[0])

Q 데이터의 크기(shape) : (11823, 40)
A 데이터의 크기(shape) : (11823, 40)

[8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [28]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs' : questions,
        'dec_inputs' : answers[:, :-1], # 디코더 입력, 마지막 패딩 토큰 제거
    },
    {
        'outputs' : answers[:, 1:] # 디코더 실제값(교사 강요), 시작 토큰 제거
    },
))

dataset = dataset.cache()
# 전처리 시간을 줄여줌
dataset = dataset.shuffle(BUFFER_SIZE)
# 데이터를 버퍼에 넣고 무작위로 빼내며 셔플, buffer_size가 데이터 크기보다 커야 함
dataset = dataset.batch(BATCH_SIZE)
# 데이터를 batch_size 개수대로 분리
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
# prefetch : 학습중일때, 데이터 로드시간을 줄이기 위해 메모리에 미리 적재시킴
# AUTOTUNE : 네트워크가 알아서 적재시킬 양을 설정해라

print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1]) # 디코더 입력 샘플
print(answers[:1][:, 1:]) # 디코더 정답 샘플

[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
[[3844   74 7894    1 8179    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


In [29]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size = VOCAB_SIZE,
    num_layers = NUM_LAYERS,
    dff = DFF,
    d_model = D_MODEL,
    num_heads = NUM_HEADS,
    dropout = DROPOUT,
    embedding_matrix = [embedding_matrix])

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y, y_hat): # metrics에 적용할 함수
  y = tf.reshape(y, shape=(-1, MAX_LENGTH - 1))
  # -1은 None과 같은 이유, 시작 토큰을 제거했으므로 길이 1 감소
  return tf.keras.metrics.sparse_categorical_accuracy(y, y_hat)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [35]:
type(embedding_matrix[0])

numpy.ndarray

In [ ]:
model.load_weights('./drive/MyDrive/models/DeeDee2_weights')

In [30]:
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 19s 54ms/step - loss: 1.4458 - accuracy: 0.0327
Epoch 2/50
185/185 [==============================] - 10s 54ms/step - loss: 1.1758 - accuracy: 0.0494
Epoch 3/50
185/185 [==============================] - 10s 54ms/step - loss: 1.0037 - accuracy: 0.0506
Epoch 4/50
185/185 [==============================] - 10s 54ms/step - loss: 0.9215 - accuracy: 0.0548
Epoch 5/50
185/185 [==============================] - 10s 56ms/step - loss: 0.8592 - accuracy: 0.0587
Epoch 6/50
185/185 [==============================] - 10s 55ms/step - loss: 0.7936 - accuracy: 0.0646
Epoch 7/50
185/185 [==============================] - 10s 55ms/step - loss: 0.7229 - accuracy: 0.0724
Epoch 8/50
185/185 [==============================] - 10s 55ms/step - loss: 0.6485 - accuracy: 0.0799
Epoch 9/50
185/185 [==============================] - 10s 56ms/step - loss: 0.5695 - accuracy: 0.0884
Epoch 10/50
185/185 [==============================] - 10s 56ms/step - loss: 0.488

In [37]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  # 첫번째 차원 추가
  
  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[: :, -1:]
    # 문제 발생
    # 해결 : 학습 자료와 다르게 단어 벡터가 세번째 축에 위치했었음
    # 슬라이싱을 세번째 축으로 함으로서 concat 연산이 올바르게 수행되게 함
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break
    
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [38]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [token for token in prediction if token < tokenizer.vocab_size])
  
  print('Input : {}'.format(sentence))
  print('Output : {}'.format(predicted_sentence))

  return predicted_sentence

In [39]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
while(True):
  sentence = input()
  if sentence == "exit":
    break
  output = predict(sentence)

In [41]:

model.save_weights('./drive/MyDrive/models/DeeDee4_weights')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(history.history.keys['acc'])